# **First Few Steps**

For the first step, we import the necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from sklearn.preprocessing import OneHotEncoder

Then, we import the dataset into this noteboook

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("drive/My Drive/Bakery Transaction Dataset/Group 3 AoL Dataset.csv")
df.head()

,Date,Time,Transaction,Item
0,2016-10-30,09:58:11,1,Bread
1,2016-10-30,10:05:34,2,Scandinavian
2,2016-10-30,10:05:34,2,Scandinavian
3,2016-10-30,10:07:57,3,Hot chocolate
4,2016-10-30,10:07:57,3,Jam


# **Data Exploration + Preprocessing**

In [ ]:
df['Item'] = df['Item'].str.lower()
df.head()

,Date,Time,Transaction,Item
0,2016-10-30,09:58:11,1,bread
1,2016-10-30,10:05:34,2,scandinavian
2,2016-10-30,10:05:34,2,scandinavian
3,2016-10-30,10:07:57,3,hot chocolate
4,2016-10-30,10:07:57,3,jam


To begin, let's see how many different items are sold in the bakery.

In [ ]:
len(df['Item'].unique())

95

There are **95** unique items that are sold in the bakery shop

In [ ]:
pd.set_option('display.max_rows', 100)
print(df['Item'].value_counts())
pd.reset_option('display.max_rows')

coffee                           5471
bread                            3325
tea                              1435
cake                             1025
pastry                            856
none                              786
sandwich                          771
medialuna                         616
hot chocolate                     590
cookies                           540
brownie                           379
farm house                        374
muffin                            370
juice                             369
alfajores                         369
soup                              342
scone                             327
toast                             318
scandinavian                      277
truffles                          193
coke                              185
spanish brunch                    172
fudge                             159
baguette                          152
jam                               149
tiffin                            146
mineral wate

We then see that there are invalid values in the dataset marked by "none". This indicates on the **cancellation** of buying a sold item. Due to that, we drop all of hte rows whose value of the 'Items' column is 'none'

In [ ]:
print(df.value_counts)

<bound method DataFrame.value_counts of              Date      Time  Transaction           Item
0      2016-10-30  09:58:11            1          bread
1      2016-10-30  10:05:34            2   scandinavian
2      2016-10-30  10:05:34            2   scandinavian
3      2016-10-30  10:07:57            3  hot chocolate
4      2016-10-30  10:07:57            3            jam
...           ...       ...          ...            ...
21288  2017-04-09  14:32:58         9682         coffee
21289  2017-04-09  14:32:58         9682            tea
21290  2017-04-09  14:57:06         9683         coffee
21291  2017-04-09  14:57:06         9683         pastry
21292  2017-04-09  15:04:24         9684      smoothies

[21293 rows x 4 columns]>


Now, we would like to see the **20** best and least selling items.

In [ ]:
print("The 20 Best Selling Items:")
print(df['Item'].value_counts().sort_values(ascending=False).head(20))
print("=================================================")
print("The 20 Least Selling Items:")
print(df['Item'].value_counts().sort_values(ascending=True).head(20))

The 20 Best Selling Items:
coffee           5471
bread            3325
tea              1435
cake             1025
pastry            856
none              786
sandwich          771
medialuna         616
hot chocolate     590
cookies           540
brownie           379
farm house        374
muffin            370
juice             369
alfajores         369
soup              342
scone             327
toast             318
scandinavian      277
truffles          193
Name: Item, dtype: int64
The 20 Least Selling Items:
bacon                            1
gift voucher                     1
raw bars                         1
polenta                          1
olum & polenta                   1
the bart                         1
adjustment                       1
chicken sand                     1
siblings                         2
spread                           2
chimichurri oil                  2
bowl nic pitt                    2
hack the stack                   2
fairy doors              

# **Running the Algorithm**

### **Data Preprocessing**

In [ ]:
df = df.drop(df[df['Item'] == 'none'].index)
print(df.value_counts)

<bound method DataFrame.value_counts of              Date      Time  Transaction           Item
0      2016-10-30  09:58:11            1          bread
1      2016-10-30  10:05:34            2   scandinavian
2      2016-10-30  10:05:34            2   scandinavian
3      2016-10-30  10:07:57            3  hot chocolate
4      2016-10-30  10:07:57            3            jam
...           ...       ...          ...            ...
21288  2017-04-09  14:32:58         9682         coffee
21289  2017-04-09  14:32:58         9682            tea
21290  2017-04-09  14:57:06         9683         coffee
21291  2017-04-09  14:57:06         9683         pastry
21292  2017-04-09  15:04:24         9684      smoothies

[20507 rows x 4 columns]>


The original dataset has 21293 rows of transactions. The processed one has 20507 rows. There should also now be 1 less unique value in the 'Items' as every row that contains the value "none" in its 'Items' column is dropped.

As the 'Date' and 'Time' columns will not be used for the algorithm as each of the transactionns are already differentiated from each other by the "Transaction" column, let's just drop them.

In [ ]:
df.head()

,Date,Time,Transaction,Item
0,2016-10-30,09:58:11,1,bread
1,2016-10-30,10:05:34,2,scandinavian
2,2016-10-30,10:05:34,2,scandinavian
3,2016-10-30,10:07:57,3,hot chocolate
4,2016-10-30,10:07:57,3,jam


In [ ]:
df = df.drop('Date', axis = 1) 
df = df.drop('Time', axis = 1)

In [ ]:
df.head()

,Transaction,Item
0,1,bread
1,2,scandinavian
2,2,scandinavian
3,3,hot chocolate
4,3,jam


We then begin conducting the hot encoding process by counting the amount of each items that are sold in each transaction.

In [ ]:
sum_df = df.groupby(['Transaction', 'Item']).size().reset_index(name='counter')
sum_df.head()

,Transaction,Item,counter
0,1,bread,1
1,2,scandinavian,2
2,3,cookies,1
3,3,hot chocolate,1
4,3,jam,1


After that, we need to combine the results into a singular dataset where each row contains the details of one single transaction, that is not repeated in the next rows. 

In [ ]:
final_df = sum_df.groupby(['Transaction', 'Item'])['counter'].sum().unstack().reset_index().fillna(0).set_index('Transaction')
final_df.head()

Item,adjustment,afternoon with the baker,alfajores,argentina night,art tray,bacon,baguette,bakewell,bare popcorn,basket,...,the bart,the nomad,tiffin,toast,truffles,tshirt,valentine's card,vegan feast,vegan mincepie,victorian sponge
Transaction,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Then, we need to make sure that the values in the now hot encoded 'Item' is only 0 or 1. The value '0' indicates that the item is **not bought** at that transaction, while the value '1' indicates that the item is **bought** at that transaction

In [ ]:
def encoding_function(a):
  if a <= 0:
    return 0
  else:
    return 1

final_df = final_df.applymap(encoding_function)
final_df.head()

Item,adjustment,afternoon with the baker,alfajores,argentina night,art tray,bacon,baguette,bakewell,bare popcorn,basket,...,the bart,the nomad,tiffin,toast,truffles,tshirt,valentine's card,vegan feast,vegan mincepie,victorian sponge
Transaction,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### **Applying the Apriori Model**

Now, let's find out the frequent itemsets using the Apriori Algorithm

In [ ]:
frequents = apriori(final_df, min_support = 0.005, use_colnames=True)

Then, let's generate the rules.

In [ ]:
pd.set_option('display.max_rows', 1000)
mined_rules = association_rules(frequents, metric='lift', min_threshold=1)
mined_rules.head(1000)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(alfajores),(coffee),0.036344,0.478394,0.019651,0.540698,1.130235,0.002264,1.135648
1,(coffee),(alfajores),0.478394,0.036344,0.019651,0.041078,1.130235,0.002264,1.004936
2,(tea),(alfajores),0.142631,0.036344,0.006762,0.047407,1.304393,0.001578,1.011614
3,(alfajores),(tea),0.036344,0.142631,0.006762,0.186047,1.304393,0.001578,1.053339
4,(jam),(bread),0.015003,0.327205,0.005071,0.338028,1.033076,0.000162,1.016349
5,(bread),(jam),0.327205,0.015003,0.005071,0.015499,1.033076,0.000162,1.000504
6,(bread),(pastry),0.327205,0.086107,0.029160,0.089119,1.034977,0.000985,1.003306
7,(pastry),(bread),0.086107,0.327205,0.029160,0.338650,1.034977,0.000985,1.017305
8,(brownie),(coffee),0.040042,0.478394,0.019651,0.490765,1.025860,0.000495,1.024293
9,(coffee),(brownie),0.478394,0.040042,0.019651,0.041078,1.025860,0.000495,1.001080


Now, let's say that we want to only display the rules that it is more likely to be correct. This means that the confidence score should be more than 50%, as it means that the rule will more likely be true than not.

In [ ]:
final_rules = mined_rules[(mined_rules['lift'] >= 1) & (mined_rules['confidence'] > 0.5)]
final_rules.sort_values('confidence', ascending = False, inplace = True)
final_rules.head(100).reset_index()

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,index,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,32,(keeping it local),(coffee),0.006656,0.478394,0.005388,0.809524,1.692169,0.002204,2.738431
1,50,(toast),(coffee),0.033597,0.478394,0.023666,0.704403,1.472431,0.007593,1.764582
2,40,(salad),(coffee),0.010460,0.478394,0.006550,0.626263,1.309094,0.001547,1.395648
3,92,"(hot chocolate, cake)",(coffee),0.011410,0.478394,0.006867,0.601852,1.258067,0.001409,1.310080
4,46,(spanish brunch),(coffee),0.018172,0.478394,0.010882,0.598837,1.251766,0.002189,1.300235
5,35,(medialuna),(coffee),0.061807,0.478394,0.035182,0.569231,1.189878,0.005614,1.210871
6,39,(pastry),(coffee),0.086107,0.478394,0.047544,0.552147,1.154168,0.006351,1.164682
7,48,(tiffin),(coffee),0.015425,0.478394,0.008452,0.547945,1.145385,0.001073,1.153856
8,0,(alfajores),(coffee),0.036344,0.478394,0.019651,0.540698,1.130235,0.002264,1.135648
9,24,(hearty & seasonal),(coffee),0.010565,0.478394,0.005705,0.540000,1.128777,0.000651,1.133926


# **Analysis of the Result**

From the results, we found that people usually buy coffee as the drink choice along as a companion to their foods.
Thus, we could say that it would be a good idea to create promotions that includes coffee in it. 

For example:
Buy 1 bread, get 1 free cup of coffee.